<img src="https://www.mines.edu/webcentral/wp-content/uploads/sites/267/2019/02/horizontallightbackground.jpg" width="100%"> 

### CSCI250 Python Computing: Building a Sensor System
<hr style="height:5px" width="100%" align="left">

# Inter-Integrated Circuit interface

# Introduction

This notebook covers the basic principles of the **Inter-Integrated Circuit interface** (I<sup>2</sup>C) and how to use it on the RPi. The only I<sup>2</sup>C device that we will use in this class is the accelerometer, so this notebook uses it as an example; the process for using other I<sup>2</sup>C devices is similar.

This notebook applies specifically to the [**MMA8452Q**](https://www.sparkfun.com/products/13926) accelerometer.

# I<sup>2</sup>C interface

The accelerometer communicates with the RPi over the protocol called [Inter-Integrated Circuit (I<sup>2</sup>C)](https://en.wikipedia.org/wiki/I%C2%B2C) (pronounced *I-squared-C*). In order to use this protocol, we need to enable the I<sup>2</sup>C interface on the RPi: go to the RPi Configuration from Preferences, then on the Interfaces tab for I<sup>2</sup>C click enable, then OK (you should have already done this from setup day).

It is not important to know all the details of how I<sup>2</sup>C works to use it, so this notebook contains just an abbreviated explanation.

All of the devices are synced together by the clock pin of the master device, denoted by SCL for **serial clock**; this clock signal is simply a square wave that repeats. Data from all devices are then sent over the data wire, denoted SDA for **serial data**. The data sent across this wire are synced up with the clock, where a high voltage on the data line indicates a 1 bit, and a low voltage indicates a 0 bit. 

Every subordinate device has an address that is assigned when the chip is manufactured; the master device is informed of the addresses by the programmer. The first data sent out by the master is the address of the device; all of the subordinate devices read the address to see if the master is communicating with them. The master can then send data to the appropriate subordinate device, and all other devices ignore these data. Once the master stops sending data, the subordinate device can send its own data to the master through the SDA wire. 

<img src="https://cdn.sparkfun.com/assets/6/4/7/1/e/51ae0000ce395f645d000000.png" width="90%">

This communication protocol may sound complicated, but it is standard, meaning that code already exists for us to use. The library we will use is called `smbus` for **System Management Bus**, and gives us very generic code to communicate with any device over I<sup>2</sup>C. This is the part where the [MMA8452Q datasheet](https://cdn.sparkfun.com/datasheets/Sensors/Accelerometers/MMA8452Q-rev8.1.pdf) is useful, because we'll need to reference specific information from it in order to use the accelerometer. Only the most vital information will be summarized here.

The first step is to import the `smbus` library with 

`import smbus`. 

Then we can create a new instance of the `smbus` object by typing 

`bus = smbus.SMBus(I2CPort)`,

where the port parameter is an integer representing the I<sup>2</sup>C bus; the RPi has 2 buses, and most often use the bus with index 1, so that is the number we use.

Note that with I<sup>2</sup>C, we don't need to specify the clock speed, because most sensors tend to just use the same speed of $100$kHz, which is considered normal speed. 

Run the code in the following cell to setup communication with the accelerometer.

In [1]:
# load the smbus library to communicate with I2C devices
import smbus

# create a new smbus object on bus index 1 of the RPi
bus = smbus.SMBus(1)

As indicated earlier, every device has an address. On page 16 of the datasheet, the first sentence in the third paragraph states the address of the accelerometer as being either `0b0011100` or `0b0011101`. The reason it gives 2 options is because you could have 2 sensors on the same I<sup>2</sup>C bus that have the same address. They would both respond at the same time causing communication issues, so you can change the address to prevent problems. We are only using 1 sensor, so we don't have to worry about it; the default address will be `0b0011101` because the last bit defaults to high. 

The binary number is useful to see when learning, but we often use hexadecimal to make writing these numbers more compact. Converting `0b0011101` to hexadecimal gives `0x1D` (note the `0x` prefix to indicate hexadecimal), which is the same value that is referenced in the datasheet.

Every I<sup>2</sup>C device has a set of registers that can either be written to or read by the master device. On page 18 of the datasheet, Table 11 includes a list of  registers that one can play with, followed by a description of their usage. Each provides different benefits, so feel free to explore and experiment with them once you're comfortable using the accelerometer. We only need a few of them to get basic functionality, so those will be described here.

Page 10 of the datasheet shows how the sensor transitions between various modes of operation. Just after the sensor is powered on, it goes into **standby mode**. As explained in Note 2 of Table 11, we can't read acceleration data in standby mode, so we need to go to the **active mode**. Page 37 of the datasheet contains information about the first control register, which allows us to put the sensor into active mode. You can see in the bit description figure that bit 0 (the last bit) is the active bit. Table 53 describes what each bit does and their default states (0 for each bit); you can see that setting the active bit to 0 sets the standby mode, and 1 sets the active mode. Obviously we want to set the active mode, so we'll need to set this bit to 1. We don't care about the rest of the options available here, and they default to 0 anyway, so we can to send the byte `0b00000001` or just 1. 
>The SMBus library has a method called `bus.write_byte_data(address, register, data)` that allows us to write data to a device. The first parameter is the address of the device, the second is the register that we want to write to, and data is what we want to write. As you can see in the datasheet, the hexadecimal value of the first control register is `0x2A`. This means the line of code we need to write to go into active mode is `bus.write_byte_data(0x1D, 0x2A, 1)`.

To quickly summarize what we did, we can't read acceleration data from the sensor until we go into active mode, so we set the active bit in one of the control registers; this is the first line of code in the cell below. That's a lot of information for a single line of code, but it is outlining the process for how to make use of an I<sup>2</sup>C sensor.

In [2]:
# set to active mode se we can read from the acceleration registers
bus.write_byte_data(0x1D, 0x2A, 1)

Now that the sensor is active, we can read the acceleration data. This sensor is a 3-axis accelerometer, meaning it measures acceleration in the $x$, $y$, and $z$ directions. Page 20 of the datasheet describes how the registers store the acceleration data: each axis has 12 bits of resolution, which are each stored in 2 registers. Because each register is 8 bits in size (1 byte), that leaves 4 extra unused bits that are always 0. If you look at the registers for the $x$ axis (`0x01` and `0x02`), the first register contains the first 8 bits, and the second contains the last 4 bits followed by 4 zeros. We'll make use of bitwise operators to combine the data into a single number.

> The SMBus library includes a method to read a set of registers sequentially from the sensor `bus.read_i2c_block_data(address, register, numBytes)`, where the first parameter is the address of the device, the second is the first register to read, and the third is the number of bytes to read (must be >0). The method returns an array of bytes from each register, so that needs to be stored somewhere. We can use this to read the data from all of the acceleration registers at once, as in the following cell. The acceleration registers start at `0x01` and there are 6 registers, so that's what we specify in the method.

In [3]:
# read data from acceleration registers
# 6 bytes total, 2 bytes for each component
data = bus.read_i2c_block_data(0x1D, 0x01, 6)

# print the data in binary and decimal formats
for i in range(6):
    print(format(data[i], "08b"),data[i])

00000000 0
10010000 144
00000000 0
01000000 64
01000001 65
00000000 0


Once we have the raw data from the registers, we can make use of the bitwise operators to make single numbers for each axis, then print out each one.

If you look on the top of the sensor, there is a printed label on the side indicating the direction of each axis, with $x$ to the right, $y$ towards you, and $z$ pointing upwards (depending on how you look at the sensor). Because Earth's gravity pulls on the sensor, we should expect to measure $9.81$m/s$^2$ on the $z$ axis and $0$ on the other axes when the sensor is flat. If you get a bunch of zeros, then you're not actually getting data from the accelerometer. There's probably an issue with your wiring somewhere or a code bug, so double check all of the setup.

In [4]:
# combine x bits from its registers
xMSB = data[0] << 4 # register 0x01 (first 8 bits of a total of 12 bits)
xLSB = data[1] >> 4 # register 0x02 ( last 4 bits of a total of 12 bits)
xRaw = xMSB + xLSB  # x data (12 bits total)

print(format(xMSB, "012b"))
print(format(xLSB, "012b"))
print(format(xRaw, "012b"), xRaw)

000000000000
000000001001
000000001001 9


In [5]:
# convert data from registers to single numbers
xRaw = (data[0] << 4) + (data[1] >> 4)
yRaw = (data[2] << 4) + (data[3] >> 4)
zRaw = (data[4] << 4) + (data[5] >> 4)

# display our data
print("xRaw", xRaw)
print("yRaw", yRaw)
print("zRaw", zRaw)

xRaw 9
yRaw 4
zRaw 1040


Let's take a look back in the datasheet to get an understanding of what we're looking at. Section 5.2 on page 12 describes the data as being 2’s complement 12 bit numbers; this means we'll need to adjust each number when it goes negative. The second paragraph states that the resolution of each data register depends on the measurement range (the datasheet calls it full scale), which can be set to $\pm$2g (1g/1024), $\pm$4g (1g/512), or $\pm$8g (1g/256).

We next need to find what the full scale of the sensor is set to. Page 22 of the datasheet describes the data configuration register, which allows us to set the full scale of the accelerometer. Table 15 states that the last 2 bits, which set the full scale, default to `0b00`. Table 16 shows how to set the bits to change the measurement range, and `0b00` corresponds to 2g. If you wanted, you could change the full scale, but now we have the information we need to decode the data that we read from the sensor!

This part is going to get a bit complicated because we need to work with binary numbers to eventually find the acceleration in each axis. We have 12 bit binary numbers in 2's complement format where the resolution is 1g/1024, so we need to do some calculations to convert these binary numbers to the number of g's in each direction.

Let's first deal with the 2's complement formatting. The numbers returned from the previous cell were all positive integers, because that's how Python interprets the binary values. It doesn't know that the number is in 2's complement, so it assumes it's just positive. Even if the binary number is `0b1111 1111 1111` making -1, it's actually just going to interpret that as 4095. Run the code in the cell below to see the binary form of each number.

In [6]:
# print numbers in binary so we can see the bits
print("xRaw", format(xRaw, "012b"), xRaw)
print("yRaw", format(yRaw, "012b"), yRaw)
print("zRaw", format(zRaw, "012b"), zRaw)

xRaw 000000001001 9
yRaw 000000000100 4
zRaw 010000010000 1040


If any of your values start with a 1 bit, that means it's a negative value. In our code we'll need to check to see if the first bit is 1 or 0. If it's 1, then we need to force the number to be a negative value. Because the number is in 2's complement format, we can do this simply by subtracting 2<sup>12</sup> (or 4096) from the number. To understand why this works, think about the edge cases: 
* if we have `0b1111 1111 1111`, that represents -1 but it's interpreted as +4095 (or 2<sup>12</sup>-1). Subtracting 4096 from 4095 gives us -1 as we want. 
* if we have `0b1000 0000 0000` which represents -2048 but is interpreted as +2048. Again subtracting 4096 gives us the correct negative value. 

To quickly recap the fix for 2's complement numbers, we need to check if the first bit is 0. If so, then we subtract 2<sup>12</sup>=4096 from the number, which makes Python give us the right number. That's the first step to adjust the numbers, next we need to scale the values as described by the datasheet. Since we are using the 2g range, that gave us a resolution of 1g/1024, so if we simply divide each number by 1024 we will get the acceleration in g's. If you instead want m/s$^2$, you can then just multiply by $9.81 m/s^2$. Then we're done! If you run the code in the following cell, you should have roughly $0$ in the $x$ and $y$ directions and $9.81 m/s^2$ in the $z$ direction.

In [7]:
# Fix for 2's complement numbers by checking what the value of the 12th digit is.
# We obtain this number by binary shift (>>) 11 times.
# If the number is negative (i.e. the remaining digit is 1), subtract 4096.
if xRaw >> 11 == 1: xRaw -= 2**12
if yRaw >> 11 == 1: yRaw -= 2**12
if zRaw >> 11 == 1: zRaw -= 2**12
    
# print numbers in binary so we can see the bits
print("xRaw", format(xRaw, "012b"), xRaw)
print("yRaw", format(yRaw, "012b"), yRaw)
print("zRaw", format(zRaw, "012b"), zRaw)

xRaw 000000001001 9
yRaw 000000000100 4
zRaw 010000010000 1040


In [8]:
# convert to acceleration
# divide by 1024 units (+/- 2g range, 1g per 1024 units)
# multiply by 9.81 for acceleration in m/s^2
toAcc = 9.81 / 1024

xAcc = xRaw * toAcc
yAcc = yRaw * toAcc
zAcc = zRaw * toAcc

# print the values in m/s^2
print("xAcc %+0.3f m/s^2"%xAcc)
print("yAcc %+0.3f m/s^2"%yAcc)
print("zAcc %+0.3f m/s^2"%zAcc)

xAcc +0.086 m/s^2
yAcc +0.038 m/s^2
zAcc +9.963 m/s^2


Hopefully everything went well and you're getting values that make sense. You can try taking measurements with the accelerometer in different orientation to see what it gives you. If you later decide to change the full scale of the accelerometer, you also need to change the scaling value as described earlier. You should have enough information to be able to use the sensor in future applications and try to make use of the other features it offers, which will require you to read about them in the datasheet.